In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urlparse

In [ ]:
# path to csv file
data = pd.read_csv("/content/drive/MyDrive/")

In [ ]:
data.head()

In [ ]:
def is_scraping_allowed(url):
    """
    Checks if scraping is allowed for the given URL.

    Args:
        url (str): The URL to check.

    Returns:
        bool: True if scraping is allowed, False otherwise.
    """
    # Parse the URL to extract the domain.
    domain = urlparse(url).netloc

    # Get the URL for the robots.txt file for the domain.
    robots_url = f"https://{domain}/robots.txt"

    # Send a GET request to the robots.txt file.
    try:
        response = requests.get(robots_url)
    except:
        # If there was an error, assume scraping is not allowed.
        return False

    # Check if the status code of the response is OK.
    if response.status_code != 200:
        # If not, assume scraping is not allowed.
        return False

    # Parse the robots.txt file.
    robots_txt = response.text

    # Check if the user-agent "*" is disallowed from scraping any pages.
    #if "User-agent: *\nDisallow: /" in robots_txt:
    #    return False

    # Check if the user-agent "*" is specifically disallowed from scraping the given URL.
    if f"User-agent: *\nDisallow: {url}" in robots_txt:
        return False

    # If none of the above conditions are met, assume scraping is allowed.
    return True

In [ ]:
def getWebsiteText(url):
    # Headers with Accept-Language set to English
    headers = {
        'Accept-Language': 'en-US,en;q=0.1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a request to the website and get the response
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            #print(f"The link {url} is working.")
            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.getText()
            return text
        else:
            return f"Error: {response.status_code}"

    except Exception as e:
        # Handle the error here
        if 'response' not in locals():
            return "Error: Failed to get response"
        return f"Error: {response.status_code}"

Check the robots.txt of each web page, by adjusting range(1751, 3500) 
-> all data: len(data)

In [ ]:
for i in range(1751, 3500):
  print(i) # only progress information
  url = data['website_url'][i]
  result = is_scraping_allowed(url)
  data.loc[i, "check_robots"] = result

In [ ]:
#Save data as new csv
data.to_csv("/content/drive/MyDrive/")

In [ ]:
data.head()

In [ ]:
#Counts True and False values in column "check_robots"
data["check_robots"].value_counts()

Gets Text from the allowed websites and saves it as txt

In [ ]:
#path for the text files 
path = "/content/drive/MyDrive/textfiles/"

In [ ]:
for i in range(1751, 3500):
  if(data["check_robots"][i]==True):
    print(i) # only progress information
    url = df_test['website_url'][i]
    text = getWebsiteText(url)
    lines = text.splitlines()
    new_lines = [line for line in lines if line]
    new_text = "\n".join(new_lines)
    # lowercase + remove everything in name but chars
    # because some names contains symbols e.g. ":" -> would destroy a textfile
    name = data["name"][i].lower()
    name = re.sub(r"[^a-z]", "", name)
    with open(path+name+".txt", "w", encoding="utf-8") as f:
      f.write(new_text)
    